# Product Environmental Footprint

This notebook is an exploration on the calculations in {cite}`zamporiSuggestionsUpdatingProduct2019`

The Circular Footprint Formula (Equation 3):

Material: $(1 -R_1)E_v + R_1\left(AE_{recycled}+(1-A)E_v*{Q_{sin}\over Q_p}\right) + (1-A)R_2\left(E_{recyclingEoL}-E^*_V{Q_{Sout} \over Q_P}\right)$

Energy: $(1-B)R_3(E_{ER}-\mathrm{LHV} X_{ER,heat}E_{SE,heat}-\mathrm{LHV} X_{ER,elev}E_{SE,elec})$

Disposal: $(1-R_2-R_3)E_D$

Annex C at: https://eplca.jrc.ec.europa.eu/permalink/Annex_C_V2.1_May2020.xlsx


In [1]:
def cff_material(r1,ev,a,erec,qsin,qp,r2,ereceol,esv,qsout):
    recycled_input = r1
    unrecycled_input = 1-r1
    recycled_output = r2
    my_burden = a
    supplier_burden = 1-a
    emission_preproc = ev
    emission_preproc_prevented = esv
    emission_recycle_in = erec
    emission_recycle_out = ereceol
    return (
        unrecycled_input*emission_preproc +
        recycled_input*my_burden      *emission_recycle_in + 
        recycled_input*supplier_burden*emission_preproc*(qsin/qp) +
        recycled_output*supplier_burden*(emission_recycle_out - emission_preproc_prevented*(qsout/qp))
    )

def cff_energy(b,r3,eer,lhv,xerheat, eseheat, xerelec,eseelec):
    return (
        (1-b)*r3*(eer - lhv*xerheat*eseheat - lhv*xerelec*eseelec)
    )

def cff_disposal(r2,r3,ed):
    return (1-r2-r3)*ed

In [2]:
import pandas as pd

PEF_ANNEX_C = "https://eplca.jrc.ec.europa.eu/permalink/Annex_C_V2.1_May2020.xlsx"

annexC_ar1r2 = pd.read_excel(PEF_ANNEX_C, "A - R1 - R2", header=[4,5,6], na_values=["-"])

# df = pd.read_excel("https://eplca.jrc.ec.europa.eu/permalink/EF3_1/EF-LCIAMethod_CF(EF-v3.1).xlsx")

ModuleNotFoundError: No module named 'pandas'

In [ ]:
annexC_ar1r2["Category"] = annexC_ar1r2["Category"].ffill()
annexC_ar1r2["Material"] = annexC_ar1r2["Material"].ffill()

# Remove unnecessary columns
annexC_ar1r2 = annexC_ar1r2.drop(axis=1, labels=[
        ("Parameters", "comments"),
        ("Parameters", "Where is the recycling rate measured"),
        ("Parameters", "Sources to define R1"),
        ("Parameters", "Sources to define R2")
    ])
# Fix column names
cols = []
for col in annexC_ar1r2.columns.values:
    col = [c for c in col if "Unnamed" not in c and "Parameters" != c]
    col = ["R2" if "Recycling rate" in c else c for c in col]
    if col[0]=="R2":
        col[1] = col[1][0:2] if ":" in col[1] else col[1]
    cols.append("-".join(col))
annexC_ar1r2.columns = cols

# Remove filler rows
annexC_ar1r2 = annexC_ar1r2[~annexC_ar1r2["Application"].isnull()]


C:\Users\yedema21\AppData\Local\Temp\ipykernel_25732\757165217.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  annexC_ar1r2 = annexC_ar1r2.drop(axis=1, labels=[


In [ ]:
annexC_ar1r2

,Category,Material,Application,A,R1,R2-EU,R2-BE,R2-BG,R2-CZ,R2-DK,...,R2-NO,R2-CH,R2-ME,R2-MK,R2-RS,R2-TR,R2-BA,R2-XK,R2-collection rate,R2-sorting
0,Metals,Steel,MATERIAL,0.2,0.00,0.85,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Metals,Steel,building - sheet,0.2,0.18,0.95,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Metals,Steel,building - stainless steel parts in copper all...,0.2,0.63,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Metals,Steel,appliances - sheet,0.2,0.18,0.9,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Metals,Steel,packaging,0.2,0.58,0.8046,0.955,0.734,0.886,0.877,...,0.877,0.86,NaN,NaN,NaN,0.57,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,Fibers,E-glass fiber,uniterruptible power supply (UPS),0.5,0.00,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
150,Fibers,Aramid,MATERIAL,0.5,0.00,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
151,Fibers,Aramid,uniterruptible power supply (UPS),0.5,0.00,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
152,Fillers,Talc filler,MATERIAL,0.5,0.00,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
